In [ ]:
%cd ../
%pip install -e .
%cd tutorials


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from poseidon.numpy import generate_rotation_matrix_with_angles

img = mpimg.imread("LARD_example.jpg")
plt.imshow(img)

df = pd.read_csv("extract_labeling_Bing.csv", delimiter=";")
dic = df.to_dict(orient="records")[0]

TL = [dic["x_TL"], dic["y_TL"]]
plt.scatter(TL[0], TL[1], label="Top Left")

TR = [dic["x_TR"], dic["y_TR"]]
plt.scatter(TR[0], TR[1], label="Top Right")

BL = [dic["x_BL"], dic["y_BL"]]
plt.scatter(BL[0], BL[1], label="Bottom Left")

BR = [dic["x_BR"], dic["y_BR"]]
plt.scatter(BR[0], BR[1], label="Bottom Right")

points_2D_LARD = torch.tensor([TL, TR, BL, BR], dtype=torch.float64).unsqueeze(0)
print("Points 2D:\n", points_2D_LARD)

plt.legend()
plt.show()

In [ ]:
import math


def geodetic_to_ecef(lat_deg, lon_deg, alt):
    # Constantes WGS84
    a = 6378137.0  # demi-grand axe (m)
    f = 1 / 298.257223563  # aplatissement
    e_sq = f * (2 - f)  # excentricité au carré

    # Conversion degrés → radians
    lat = math.radians(lat_deg)
    lon = math.radians(lon_deg)

    # Rayon de courbure en N
    N = a / math.sqrt(1 - e_sq * math.sin(lat) ** 2)

    # Coordonnées ECEF
    x = (N + alt) * math.cos(lat) * math.cos(lon)
    y = (N + alt) * math.cos(lat) * math.sin(lon)
    z = (N * (1 - e_sq) + alt) * math.sin(lat)

    return x, y, z

In [ ]:
# Rotation
yaw, pitch, roll = dic["yaw"], dic["pitch"], dic["roll"]
print("(yaw,pitch,roll):", yaw, pitch, roll)
R = torch.tensor(
    generate_rotation_matrix_with_angles(yaw, pitch, roll), dtype=torch.float64
).unsqueeze(0)
print("Rotation matrix (R):")
print(R, R.shape)


# Position
x, y, z = geodetic_to_ecef(dic["lat_cam"], dic["lon_cam"], dic["alt_cam"])
C = torch.tensor([[x], [y], [z]], dtype=torch.float64).unsqueeze(0)
print("Camera position (C):")
print(C, C.shape)

# Intraseca
A = torch.tensor([[60, 0, 0], [0, 60, 0], [0, 0, 1]], dtype=torch.float64).unsqueeze(0)
print("Camera intrinsic parameters (A):")
print(A, A.shape)

In [ ]:
json_str = """{
  "CYEG": {
    "20": {
      "A": {
        "position": {
          "x": -1523982.3717219266,
          "y": -3488778.151048129,
          "z": 5109273.396875209
        },
        "coordinate": {
          "latitude": 53.30882382781787,
          "longitude": -113.59686497811173,
          "altitude": 706.4014546920321
        }
      },
      "B": {
        "position": {
          "x": -1523948.9206793625,
          "y": -3488822.6676147007,
          "z": 5109253.674062423
        },
        "coordinate": {
          "latitude": 53.30852026046496,
          "longitude": -113.59613546618918,
          "altitude": 706.9605077653717
        }
      },
      "C": {
        "position": {
          "x": -1521316.939711129,
          "y": -3487577.1019060216,
          "z": 5110895.239647287
        },
        "coordinate": {
          "latitude": 53.33316364003713,
          "longitude": -113.56732397849756,
          "altitude": 712.6417539967418
        }
      },
      "D": {
        "position": {
          "x": -1521350.0394275817,
          "y": -3487532.799066956,
          "z": 5110914.564752514
        },
        "coordinate": {
          "latitude": 53.33346484967192,
          "longitude": -113.56804756763657,
          "altitude": 711.7967542711499
        }
      }
    }
  }
}"""

In [ ]:
import json

import numpy as np
import torch

# Charge le JSON en dict Python
data = json.loads(json_str)

# Récupère les positions des points A, B, C, D
positions = []
for key in ["B", "A", "C", "D"]:
    pos = data["CYEG"]["20"][key]["position"]
    positions.append([pos["x"], pos["y"], pos["z"]])

posi = []
for key in ["B", "A", "C", "D"]:
    pos = data["CYEG"]["20"][key]["coordinate"]
    a, b, c = geodetic_to_ecef(pos["latitude"], pos["longitude"], pos["altitude"])
    posi.append([a, b, c])
print("x,y,z positions:", posi)
print("ECEF positions::", positions)
# Conversion en numpy array et torch tensor
positions_np = np.array(positions)
points_3D = torch.tensor(positions_np, dtype=torch.float64).unsqueeze(0)

print("3D points (torch) : \n", points_3D, points_3D.shape)

from poseidon.torch.utils.before_p3p import projection_all_point3D_to2D

points_2D = projection_all_point3D_to2D(points_3D, C, R, A)
print("2D points (torch) : \n", points_2D, points_2D.shape)
print("2D", points_2D_LARD)

In [ ]:
import torch
from poseidon.torch import compute_features_vectors

features_vect = compute_features_vectors(points_3D, C, R)
print("Feature vectors:\n", features_vect, features_vect.shape)


import torch


def compute_feature_vectors_from_2D(points_2D, A):
    fet = []

    fx = A[0, 0, 0]
    fy = A[0, 1, 1]
    cx = A[0, 0, 2]
    cy = A[0, 1, 2]

    print("fx, fy, cx, cy:", fx, fy, cx, cy)

    for i in range(3):
        # points_2D shape: [1, 3, 2], so points_2D[0, i] is [u, v]
        u, v = points_2D[0, i]

        # Normalize using intrinsics
        f = torch.tensor([(u - cx) / fx, (v - cy) / fy, 1.0], dtype=torch.float64)

        # Normalize vector to make it unitary
        f = f / torch.linalg.norm(f)

        fet.append(f)

    return torch.stack(fet, dim=0)  # Shape: [3, 3]


# ft = compute_feature_vectors_from_2D((points_2D[:,:3]),A).unsqueeze(0)
# print("Feature vectors from 2D points:\n", ft)

In [ ]:
# Create of the 3D figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(C[:, 0].squeeze(), C[:, 1].squeeze(), C[:, 2].squeeze(), color="orange")

for i in range(3):
    Pi = points_3D[:, i].squeeze()
    ax.scatter(*Pi, color="black")
    ax.text(*Pi, f"$P_{i+1}$")
    ax.plot(
        [C[:, 0].squeeze(), Pi[0]], [C[:, 1].squeeze(), Pi[1]], [C[:, 2].squeeze(), Pi[2]], "k--"
    )
    f = Pi - C.squeeze()
    ax.quiver(
        C[:, 0].squeeze(),
        C[:, 1].squeeze(),
        C[:, 2].squeeze(),
        f[0],
        f[1],
        f[2],
        color="red",
        normalize=True,
    )
    ax.text(
        (C[:, 0].squeeze() + Pi[0]) / 2,
        (C[:, 1].squeeze() + Pi[1]) / 2,
        (C[:, 2].squeeze() + Pi[2]) / 2,
        f"$f_{i+1}$",
        color="red",
    )


# Set axis limits to visualize a 4x4x4 space centered around zero

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("The three 3D Points and features vectors use for P3P")

ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from poseidon.torch import P3P, find_best_solution_P3P_batch

solutions = P3P(points_3D[:, 1:], features_vect)
print("Solutions from P3P:\n", solutions)

R_solution, C_solution, estimation_error = find_best_solution_P3P_batch(
    solutions, points_2D, points_3D, A
)

print("\033[1m Estimated Rotation Matrix \033[0m (R_solution):\n ", R_solution)
print("\033[1m Original Rotation Matrix \033[0m  (R):\n", R, "\n")
print(
    "\033[1m Estimated Camera Position \033[0m  (C_solution):\n",
    C_solution,  # (-R_solution) @ C_solution # si R shape: [B, 3, 3], t: [B, 3]*
)
print("\033[1m Original Camera Position \033[0m (C):\n", C, "\n")
print("Estimation Error:\n", np.linalg.norm(C - C_solution))

In [ ]:
from poseidon.numpy import find_best_solution_P3P, solve_reformat_p3p_solutions

points_2D_LARD_np = points_2D_LARD.squeeze(0).numpy()
A_np = A.squeeze(0).numpy()
print(positions_np[:3].shape, A_np.shape)
sol = solve_reformat_p3p_solutions(positions_np[:3], points_2D_LARD_np[:3], A_np)
R_opencv, C_opencv, err = find_best_solution_P3P(points_2D_LARD_np, positions_np, sol, A_np)
print("\033[1m OpenCV Estimated Rotation Matrix \033[0m (R_opencv):\n ", R_opencv)
print("\033[1m OpenCV Original Rotation Matrix \033[0m  (R_solution):\n", R_solution, "\n")
print(R)
print("\033[1m OpenCV Estimated Camera Position \033[0m  (C_opencv):\n", C_opencv)
print("\033[1m Original Camera Position \033[0m (C_solution):\n", C_solution, "\n")
print(C)
print("Estimation Error OpenCV:\n", err)
print("Estimation Error torch:\n", estimation_error)